In [1]:
import pandas as pd
import numpy as np
import json
import re

In [2]:
df=pd.read_csv('Ashutosh.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Title          219 non-null    object
 1   JSON Response  218 non-null    object
dtypes: object(2)
memory usage: 3.6+ KB


In [5]:
df.isnull().sum()

Title            4
JSON Response    5
dtype: int64

In [6]:
df.dropna(inplace=True)

In [7]:
df = df.dropna(axis=1, how='all')

In [8]:
df.shape

(217, 2)

In [9]:
def is_valid_json(json_str):
    try:
        json.loads(json_str)
        return True
    except Exception as err:
        print("exception occured",err)
        return False

df = df[df['JSON Response'].apply(is_valid_json)].reset_index(drop=True)
df.shape

(217, 2)

In [10]:
df.head()

,Title,JSON Response
0,Django,"{\n ""id"": 1250569,\n ""name"": ""Django"",\n..."
1,Coding: Intermediate level,"{\n ""id"": 1471468,\n ""name"": ""Coding: In..."
2,Docker,"{\n ""id"": 170977,\n ""name"": ""Docker"",\n ..."
3,ASP.NET,"{\n ""id"": 895134,\n ""name"": ""ASP.NET (in..."
4,Creating REST APIs,"{\n ""id"": 1950355,\n ""name"": ""Creating R..."


In [11]:
def extract_preview_questions(json_str):
    try:
        parsed = json.loads(json_str)
        preview_questions = parsed.get("preview_questions", None)
        return preview_questions
    except Exception:
        return None
        
df['preview_questions'] = df['JSON Response'].apply(extract_preview_questions)

In [12]:
df.head()

,Title,JSON Response,preview_questions
0,Django,"{\n ""id"": 1250569,\n ""name"": ""Django"",\n...","[{'id': 1095950, 'text': '<p>You are working o..."
1,Coding: Intermediate level,"{\n ""id"": 1471468,\n ""name"": ""Coding: In...","[{'id': 1278402, 'text': '<p><span style=""back..."
2,Docker,"{\n ""id"": 170977,\n ""name"": ""Docker"",\n ...","[{'id': 155197, 'text': '<p>Given the Dockerfi..."
3,ASP.NET,"{\n ""id"": 895134,\n ""name"": ""ASP.NET (in...","[{'id': 615710, 'text': '<p>Which code snippet..."
4,Creating REST APIs,"{\n ""id"": 1950355,\n ""name"": ""Creating R...","[{'id': 1561619, 'text': '<p><span style=""back..."


In [13]:
# df['preview_questions'].iloc[0]

In [14]:
def clean_and_format_questions(preview_questions):
    formatted_questions = []

    for question in preview_questions:
        # Extract the question text (title)
        title = question.get("text", "").strip()

        # Map the question type to an integer
        ques_type = 1  # Default to multiple-choice
        if question.get("type") == "true-false":  # Example for other types
            ques_type = 2

        # Extract options and identify the correct answer
        options = []
        correct_option = None
        for answer in question.get("answers", []):
            options.append(answer.get("text", "").strip())
            if answer.get("score", 0) == 5:  # Assuming 5 is the score for the correct answer
                correct_option = answer.get("text", "").strip()

        # Add the formatted question to the list
        formatted_questions.append({
            "title": title,
            "ques_type": ques_type,
            "options": options,
            "score": correct_option
        })

    return formatted_questions

In [15]:
df.head()

,Title,JSON Response,preview_questions
0,Django,"{\n ""id"": 1250569,\n ""name"": ""Django"",\n...","[{'id': 1095950, 'text': '<p>You are working o..."
1,Coding: Intermediate level,"{\n ""id"": 1471468,\n ""name"": ""Coding: In...","[{'id': 1278402, 'text': '<p><span style=""back..."
2,Docker,"{\n ""id"": 170977,\n ""name"": ""Docker"",\n ...","[{'id': 155197, 'text': '<p>Given the Dockerfi..."
3,ASP.NET,"{\n ""id"": 895134,\n ""name"": ""ASP.NET (in...","[{'id': 615710, 'text': '<p>Which code snippet..."
4,Creating REST APIs,"{\n ""id"": 1950355,\n ""name"": ""Creating R...","[{'id': 1561619, 'text': '<p><span style=""back..."


In [16]:
# df['preview_questions'].iloc[0]

In [17]:
def map_question_type(question_type, answers):
    if question_type == "multiple-choice":
        correct_answers = [ans for ans in answers if ans.get("score", 0) == 5]
        if len(correct_answers) > 1:
            return 15
        else:
            return 2
    elif question_type == "true-false":
        return 11
    else:
        return None

In [18]:
def extract_correct_answers(answers, ques_type):
    """
    Extracts the correct answer(s) based on the question type.
    """
    if ques_type in [2, 11]:
        # Single correct answer
        correct_answer = next((ans["text"] for ans in answers if ans.get("score", 0) == 5), None)
        return clean_html_tags(correct_answer) if correct_answer else None
    elif ques_type == 15:
        # Multiple correct answers
        correct_answers = [clean_html_tags(ans["text"]) for ans in answers if ans.get("score", 0) == 5]
        return correct_answers if correct_answers else None
    else:
        return None

In [19]:
def clean_html_tags(text):
    """
    Removes HTML tags from the input text.
    """
    if not text:
        return text
    # Remove HTML tags using regex
    clean_text = re.sub(r"<[^>]+>", "", text)
    # Replace HTML entities (e.g., &nbsp;, &amp;)
    clean_text = clean_text.replace("&nbsp;", " ").replace("&amp;", "&")
    return clean_text.strip()

In [20]:
def clean_and_format_questions(preview_questions):
    formatted_questions = []

    for question in preview_questions:
        # Clean the question text
        title = clean_html_tags(question.get("text", ""))

        # Map the question type
        ques_type = map_question_type(question.get("type"), question.get("answers", []))
        if not ques_type:
            continue  # Skip unsupported question types

        # Extract and clean options
        options = [clean_html_tags(ans.get("text", "")) for ans in question.get("answers", [])]

        # Extract the correct answer(s)
        score = extract_correct_answers(question.get("answers", []), ques_type)

        # Add the formatted question to the list
        formatted_questions.append({
            "title": title,
            "ques_type": ques_type,
            "options": options,
            "score": score
        })

    return formatted_questions

In [21]:
formatted_questions = clean_and_format_questions(preview_questions)
print(formatted_questions)

NameError: name 'preview_questions' is not defined